<a href="https://colab.research.google.com/github/ahorvath008/Coursera_Capstone/blob/main/Toronto_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import seaborn as sns
import scipy as stats

In [68]:
#Webscraping the data from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header=0)
df = df[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [69]:
#Printing the shape of the dataframe
df.shape

(180, 3)

In [82]:
#df["Postal Code"] = df["Postal Code"].astype(str)
#df["Postal Code"].str.replace('\n', '')
#df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [86]:
#Data cleaning by dropping the "Not Assigned" boroughs. 
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [ ]:
#Import the Data from the Coordinates file
df1 = pd.read_csv("Toronto.csv")
df1.head(5)

In [88]:
#Combine the two dataframes
df_combined = pd.merge(df, df1, on='Postal Code')
df_combined.head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [89]:
#Validate the total lines within the file
df.shape

(103, 3)

In [ ]:
#Explore data

In [101]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [102]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [113]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Neighbourhood in zip(df_combined['Latitude'], df_combined['Longitude'], df_combined['Neighbourhood']):
    label = '{}'.format(Neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

In [135]:
#Define Foursquare credentials

CLIENT_ID = 'WD0FC5WVUL3SS404ZVHJEPFGW4L310PRJ51ZHSIGJFB1M4D1' # your Foursquare ID
CLIENT_SECRET = 'TSOOUBQLXPK5KJQ1GKKOMASD1U35FNO0IW1WRHWYPIN5IOX3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: WD0FC5WVUL3SS404ZVHJEPFGW4L310PRJ51ZHSIGJFB1M4D1
CLIENT_SECRET:TSOOUBQLXPK5KJQ1GKKOMASD1U35FNO0IW1WRHWYPIN5IOX3


In [120]:
#We will select a neighborhood in Toronto
df_combined.loc[10, 'Neighbourhood']

'Glencairn'

In [121]:
neighborhood_latitude = df_combined.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_combined.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_combined.loc[10, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Glencairn are 43.709577, -79.44507259999999.


In [138]:
#Explore venues around the selected location in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [139]:
# Now run the function for the dataset
Toronto_venues = getNearbyVenues(names=df_combined['Neighbourhood'],
                                  latitudes=df_combined['Latitude'],
                                  longitudes=df_combined['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [142]:
#Check the size of the dataframe
print(Toronto_venues.shape)
Toronto_venues.head()

(135, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
1,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
2,"Regent Park, Harbourfront",43.654260,-79.360636,Jane Roos Gallery,43.653962,-79.361090,Art Gallery
3,"Regent Park, Harbourfront",43.654260,-79.360636,King Deli Cafe,43.653906,-79.361147,Sandwich Place
4,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Roots,43.718214,-79.463893,Boutique


In [143]:
#Let's check how many venues are there per neighbourhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",3,3,3,3,3,3
"Bedford Park, Lawrence Manor East",3,3,3,3,3,3
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",1,1,1,1,1,1
Central Bay Street,3,3,3,3,3,3
"Commerce Court, Victoria Hotel",16,16,16,16,16,16
Davisville,9,9,9,9,9,9
Don Mills,3,3,3,3,3,3
"Fairview, Henry Farm, Oriole",2,2,2,2,2,2
"First Canadian Place, Underground city",12,12,12,12,12,12


In [149]:
# Analyze each neighborhood
# one hot encoding

toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Food Court,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Hot Dog Joint,Hotel,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Middle Eastern Restaurant,Park,Performing Arts Venue,Pharmacy,Playground,Pool,Pub,Record Shop,Restaurant,Roof Deck,Salad Place,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Lawrence Manor, Lawrence Heights",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [150]:
#Identify the frequency of occurence within each category of establishment
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Food Court,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Hot Dog Joint,Hotel,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Liquor Store,Middle Eastern Restaurant,Park,Performing Arts Venue,Pharmacy,Playground,Pool,Pub,Record Shop,Restaurant,Roof Deck,Salad Place,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.333333,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
1,"Bedford Park, Lawrence Manor East",0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000,0.0,0.0,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
3,Central Bay Street,0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.333333,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,"Commerce Court, Victoria Hotel",0.0625,0.0625,0.000000,0.0625,0.0,0.000000,0.000000,0.0,0.000000,0.0625,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.062500,0.000000,0.0,0.062500,0.0,0.000000,0.000000,0.062500,0.000000,0.000000,0.0,0.062500,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0625,0.000000,0.000000,0.000000,0.062500,0.062500,0.062500,0.0625,0.00,0.000000,0.062500,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
5,Davisville,0.0000,0.0000,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,

In [156]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.33
1              Deli / Bodega  0.33
2        Fried Chicken Joint  0.33
3  Middle Eastern Restaurant  0.00
4                        Pub  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.33
1    Sushi Restaurant  0.33
2  Italian Restaurant  0.33
3                 Pub  0.00
4                Pool  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                   venue  freq
0  Performing Arts Venue   1.0
1    American Restaurant   0.0
2       Greek Restaurant   0.0
3                    Pub   0.0
4                   Pool   0.0


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.33
1             Sandwich Place  0.33
2                   Pharmacy  0.33
3  Middle Eastern Restaurant  0.00
4              

In [160]:
num_top_venues = 10

# write a funcition to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)   
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_venues_sorted = pd.DataFrame(columns=columns)
Toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    Toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Fried Chicken Joint,Deli / Bodega,Yoga Studio,Dessert Shop,Cocktail Bar,College Gym,Concert Hall,Cupcake Shop,Diner
1,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Italian Restaurant,Coffee Shop,Cocktail Bar,College Gym,Concert Hall,Cupcake Shop,Deli / Bodega,Dessert Shop,Yoga Studio
2,"CN Tower, King and Spadina, Railway Lands, Har...",Performing Arts Venue,Yoga Studio,Chinese Restaurant,Coffee Shop,College Gym,Concert Hall,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner
3,Central Bay Street,Pharmacy,Coffee Shop,Sandwich Place,Yoga Studio,Deli / Bodega,Cocktail Bar,College Gym,Concert Hall,Cupcake Shop,Dessert Shop
4,"Commerce Court, Victoria Hotel",American Restaurant,Bookstore,Burrito Place,Coffee Shop,Pub,Deli / Bodega,Salad Place,Sandwich Place,Seafood Restaurant,Soup Place
5,Davisville,Dessert Shop,Café,Italian Restaurant,Toy / Game Store,Coffee Shop,Seafood Restaurant,Sushi Restaurant,Cocktail Bar,College Gym,Concert Hall
6,Don Mills,Restaurant,Baseball Field,Pool,Yoga Studio,Deli / Bodega,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Cupcake Shop
7,"Fairview, Henry Farm, Oriole",Theater,Bank,Yoga Studio,Diner,Coffee Shop,College Gym,Concert Hall,Cupcake Shop,Deli / Bodega,Dessert Shop
8,"First Canadian Place, Underground city",Gluten-free Restaurant,Salad Place,Gym / Fitness Center,Deli / Bodega,Cupcake Shop,Coffee Shop,Café,Gastropub,Burger Joint,Building
9,"Forest Hill North & West, Forest Hill Road Park",Furniture / Home Store,Yoga Studio,Dessert Shop,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Cupcake Shop,Deli / Bodega,Diner


In [169]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 2, 0, 2, 2, 2, 2, 2, 2], dtype=int32)

In [ ]:
# The clustering is not working fully as the indeces are of different lenght.  I was not able to fix

toronto_merged = df_combined

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()